<div style="text-align: center"><img src="https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F1cd5fbcb-a8fb-4ac6-8b7e-e56ad38d8bf3_1644x1148.png" width="100%" heigh="100%" alt="Retrieve&Re-Rank pipeline"></div>

Data storytelling is a powerful tool for communicating development data to the world.

In [1]:
import pandas as pd

# Load and preprocessthe dataset
data = pd.read_csv('/kaggle/input/un-food-security/FAOSTAT_data_en_2-23-2024.csv')

# Adjusted preprocess for the 'Year' column to accurately represent the middle year of the range
def get_middle_year(year_range):
    start_year, end_year = map(int, year_range.split('-'))
    # Calculate middle year correctly as per the given instruction
    return start_year + (end_year - start_year) // 2

data['Year'] = data['Year'].apply(get_middle_year)

# Preprocess the 'Value' column to remove any comparison operators and convert to float
def clean_value(value):
    if isinstance(value, str):
        if value.startswith('<') or value.startswith('>'):
            # Assume values like '<0.1' to be '0.1' for simplicity
            return float(value[1:].strip())
        return float(value.strip())
    return value

data['Value'] = data['Value'].apply(clean_value)

And then we can do a bit of fancy footwork with Python code — to create a Pandas data frame that we can use to visualize our data.

# Story 1. Global Prevalence of Undernourishment

For this exercise, we want to find the real story about the levels of undernourishment around the globe. Higher levels of undernourishment are a pretty clear indicator that food security in a particular area is low.

With our data set, to analyze the “Prevalence of undernourishment” indicator across all countries, we can first aggregate the data to calculate the global average prevalence of undernourishment per year.

This involves filtering the dataset for the specific indicator, converting the Year column to a numeric format for easier analysis, and then calculating the mean value of undernourishment for each year across all countries.

**Credit**:

https://johnloewen.substack.com/p/data-storytelling-with-python-from?utm_source=profile&utm_medium=reader2